# Welcome to the Motion Gestures Toolbox (Python) - Tutorial/Documentation

## Video visualisation
Videos can be watched as they are, but they can also be used to develop new visualisations to be used for analysis. The aim of creating such alternate displays from video recordings is to uncover features, structures and similarities within the material itself, and in relation to, for example, score material. Three useful visualisation techniques here are motion images, motion history images and motiongrams.

MGT can generate both dynamic and static visualizations, as well as some quantitative data:

- dynamic visualisations (video files)
    - motion video
    - motion history video
- static visualisations (images)
    - motion average image
    - motiongrams
    - videograms
- motion data (csv files)
    - quantity of motion
    - centroid of motion
    - area of motion

In the following we will try this ourselves, and look at the different types.

## Dependencies

To make sure you have all the necessary dependencies, evaluate the following line in the terminal:

`pip3 install numpy pandas matplotlib opencv-python moviepy ffmpeg ffmpeg-python scipy`


## Import

If you have all the dependencies installed, go ahead and import the `mgmodule`.

In [2]:
import mgmodule

## The MgObject

### Simple video import

Now we create our mg (motion gestures) object. 

You can simply read a video file from the current directory this way:

In [ ]:
mg = mgmodule.MgObject('dance.avi') # from the current directory

Relative paths also work. Here is _pianist.avi_ from the __examples__ (sibling) directory:

In [ ]:
mg = mgmodule.MgObject('./examples/pianist.avi') # from a sibling directory

With absolute path:

In [ ]:
import os
abs_path = os.path.abspath('dance.avi')
print(f"The absolute path to dance.avi is '{abs_path}'.")
mg = mgmodule.MgObject(abs_path) # as absolute path

You can watch your video with calling the `show()` method:

In [ ]:
mg.show() # press q to quit video

### A brief note on containers and codecs
We aim to provide `mgmodule` as a cross-platform, flexible toolbox. Unfortunately different video containers and codecs are not equally well-supported across different containers, API-s, and operating systems. Our current solution to ensure that our toolbox works for you regardless of all that is to use the `mjpeg` codec with the `avi` container. Therefore all imported videos which don't match these are automatically converted (using ffmpeg) as a 0th step, and all subsequent processes are executed on the resulting files.

## Preprocessing modules
### Trimming
When creating the object you can also already apply some preprocessing. For example you can trim the duration of the video like this:

In [ ]:
# the numbers used for starttime and endtime represent time from the video's timeline in seconds
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15)
mg.show() # view the result, press q to quit video

This will create the file *dance_trim.avi* in the same directory.

### Skipping
In order to save time, skipping every other frame, or more, in the analysis can give you a faster analysis while still getting an idea of the motion. You can for example set this by adding `skip=2`, to skip two frames before including a frame in the analysis, then skipping two again. 

In [ ]:
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15, skip=2)
mg.show()

This will create the file *dance_trim_skip.avi* in the same directory. Notice how the added suffixes at the end of the file's name can inform you about the processes the material went through. 

### It's a chain
It is also worth to note that the preprocessing modules work as a chain (- more on that below). In this case that means we first load the video file, then trim its start to 5s and its end to 15s. Then we take the resulting *dance_trim.avi* and discard 2 out of every 3 frames. (Keeping the 1st, skipping 2nd and 3rd, keeping the 4th, skipping 5th and 6th, and so on...) The resulting file of this process is *dance_trim_skip.avi*.

### Rotating

Sometimes source videos are recorded with a slightly off horizon, or with the camera mounted sideways, therefore it is desirable to rotate the video by a few (or more) degrees. We can do this by simply specifying the angle we want to rotate with for the `rotate` parameter of our MgObject:

In [ ]:
# rotate by 90 degrees
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15, skip=3, rotate=90)
mg.show()

In [ ]:
# or just a little bit...
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15, skip=3, rotate=5.31)
mg.show()

Again, the resulting filename *dance_trim_skip_rot.avi* will inform us about the chain of processes *dance.avi* went through.

### Adjusting contrast and brightness
During preprocessing you can also add (or remove) some contrast and brightness of your video.

Let's crank up the contrast and brighten up our *dance.avi*!

In [ ]:
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15, skip=3, rotate=90, contrast=100, brightness=20)
mg.show()

The resulting file is now called *dance_trim_skip_rot_cb.avi*.

### Cropping

If the video frame has big areas with no motion occuring, a lot of time could be saved if only the area with motion was used in the analysis. One useful tool developed for the pre-analysis is the crop = 'auto' input, which automatically finds the area with significant motion in the input video. The movement occuring has to be above a low threshold, as to not include irrelevant background motion from shadows, dust etc. Another mode is crop = 'manual', where you can manually mark a rectangle around your area of interest.

#### Automatic cropping

In [ ]:
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15, skip=3, rotate=90, contrast=100, brightness=20, crop='auto')
mg.show()

#### Manual cropping

In [ ]:
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15, skip=3, rotate=90, contrast=100, brightness=20, crop='manual')
mg.show()

The resulting file is now called *dance_trim_skip_rot_cb_crop.avi*.

### Grayscale mode

So far all of our work preserved the color space of the source video. At the final preprocessing stage we can also choose to convert the video to grayscale with specifying `color=False` (by default `color=True`). This will not only result in a grayscale version of our source video, but also informs all future processes to work in grayscale mode. The technical benefit of this can be slightly shorter processing times at most processes, since in grayscale mode we process a single color channel per frame (instead of three channels). Try this:

In [ ]:
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15, skip=3, rotate=90, contrast=100, brightness=20, crop='auto', color=False)
mg.show()

The resulting file is now called *dance_trim_skip_rot_cb_crop_gray.avi*.

### Summary of preprocessing modules
As we have seen, we can optionally apply six types of preprocessing to the video we load into our MgObject, they are (in order of execution):

- trim: Trim contents of the video based on `starttime` and `endtime`.
- skip: Skip every n frames, where n is determined by `skip`.
- rotate: Rotate the video by an angle determined by `rotate`.
- cb: Adjust contrast and brightness of the video, where `contrast` and `brightness` are the level of adjustment in percentages (meaning `contrast=0` will not apply any change). both values range from `-100` to `100`.
- crop: Crop frames in video. If `crop='auto'` the module will attempt to find the area of motion, if `crop='manual'` we can draw the cropping rectangle over the first frame.
- grayscale: Convert the video to grayscale with specifying `color=False`. This will also cause all further processes called on the MgObject to function in grayscale mode.


### Keep everything
Notice that although we can optionally apply up to four preprocessing modules to our source video, normally we only keep the final result. If you would like to keep the results of all modules, set `keep_all=True`.

In [ ]:
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15, skip=3, rotate=90, contrast=100, brightness=20, crop='auto', color=False, keep_all=True)

This will output six new video files:
- *dance_trim.avi*
- *dance_trim_skip.avi*
- *dance_trim_skip_rot.avi*
- *dance_trim_skip_rot_cb.avi*
- *dance_trim_skip_rot_cb_crop.avi*
- *dance_trim_skip_rot_cb_crop_gray.avi*

## Processes

In the following we will take a look at several functions to further process our videos. These include:
- `motion()`: The most frequently used function, generates a *_motion* video, horizontal and vertical motiongrams, and plots about the centroid and quantity of motion found in the video.
- `history()`: Generates a *_history* video by layering the last n frames on the current frame for each frame in the video.
- `motionhistory()`: Generates a *_motionhistory* video, which is equivalent to calling `history()` on a *_motion* video.
- `average()`: Generates an *_average* image of all frames in the video.
- `flow.sparse()`: Generates a *_sparse* optical flow video.
- `flow.dense()`: Generates a *_dense* optical flow video. 

### Motion analysis

By calling the `motion()` function, we will generate a number of files from the input video, in the same location as the source file.

These include:
- *<input_filename>_motion.avi*: The motion video that is used as the source for the rest of the analysis.
- *<input_filename>_mgx.png*: A horizontal motiongram.
- *<input_filename>_mgy.png*: A vertical motiongram.
- *<input_filename>_motion_com_qom.png*: An image file with plots of centroid and quantity of motion

We will examine each of these in a little more detail.

In [ ]:
import mgmodule
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15, skip=1, contrast=100, brightness=20, crop='auto')
mg.motion()

In [ ]:
ls # take a look at the output files

We can now look at the results with using the `key` parameter of `show()`.

In [ ]:
mg.show(key='motion') # show the motion video of the preprocessed input, in this case 'dance_trim_skip_cb_crop_motion.avi'

In [ ]:
mg.show(key='mgx') # show the horizontal motiongram, here 'dance_trim_skip_cb_crop_mgx.png'

In [ ]:
mg.show(key='mgy') # show the vertical motiongram

In [ ]:
mg.show(key='plot') # show the image of the two plots ('Centroid of motion' and 'Quantity of motion') also shown at the end of motion()

Alternatively we can display the images right here in our notebook:

In [ ]:
from IPython.display import Image
x = Image('dance_trim_skip_cb_crop_mgx.png')
x

In [ ]:
y = Image('dance_trim_skip_cb_crop_mgy.png')
y

In [ ]:
com_qom = Image('dance_trim_skip_cb_crop_motion_com_qom.png')
com_qom

You can also configure `motion()` to output only the files you need:

In [ ]:
import mgmodule
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15, skip=1, contrast=100, brightness=20, crop='auto')

In [ ]:
# without plot
mg.motion(save_plot=False)

In [ ]:
# without plot and data
mg.motion(save_plot=False, save_data=False)

In [ ]:
# without plot, data and motiongrams (so only the video)
mg.motion(save_plot=False, save_data=False, save_motiongrams=False)

In [ ]:
# without video, plot and motiongrams (so only the data)
mg.motion(save_plot=False, save_video=False, save_motiongrams=False)

When it comes to the motion data, you can choose from several different formats:

In [ ]:
import mgmodule
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15, skip=1, contrast=100, brightness=20, crop='auto')

In [ ]:
# save only the data as .csv (default)
mg.motion(save_plot=False, save_motiongrams=False, save_video=False, data_format="csv")

In [ ]:
# save only the data as .tsv
mg.motion(save_plot=False, save_motiongrams=False, save_video=False, data_format="tsv")

In [ ]:
# save only the data as .txt
mg.motion(save_plot=False, save_motiongrams=False, save_video=False, data_format="txt")

In [ ]:
# saving in multiple formats if data_format is a list
mg.motion(save_plot=False, save_motiongrams=False, save_video=False, data_format=["txt", "csv"])

#### Filtering types
If you think there is too much noise in the output images or video, you may choose to use some other filter settings.

Filtertypes availible are:

- `Regular` turns all values below `thresh` to 0.
- `Binary` turns all values below `thresh` to 0, above thres to 1.
- `Blob` removes individual pixels with erosion method.

Try this:

In [ ]:
mg.motion(filtertype='Blob')
mg.show(key='motion')

#### Effects of filtering

Finding the right `thresh`old value is crucial for accurate motion extraction. Let's see a few examples.

First we import the requirements for the rest of the code.

In [ ]:
import mgmodule
from IPython.display import Image

Then we import the example video.

In [ ]:
mg = mgmodule.MgObject('dance.avi', starttime=3, endtime=15, skip=0, contrast=100, brightness=20)

First we can try to run without any threshold. This will result in a result in which much of the background noise will be visible, including traces of keyframes if the video file has been compressed.

In [ ]:
mg.motion(thresh=0)
x = Image('dance_trim_cb_mgy.png')
x

Adding just a little bit of thresholding (0.02 here) will drastically improve the final result.

In [ ]:
mg.motion(thresh=0.02)
x = Image('dance_trim_cb_mgy.png')
x

The standard threshold value (0.1) generally works well for many types of videos.

In [ ]:
mg.motion(thresh=0.1)
x = Image('dance_trim_cb_mgy.png')
x

A more extreme value (for example 0.5) will remove quite a lot of the content, but may be useful in some cases with very noisy videos.

In [ ]:
mg.motion(thresh=0.5)
x = Image('dance_trim_cb_mgy.png')
x

As the above examples have shown, choosing the thresholding value is important for the final output result. While it often works to use the default value (0.1), you may improve the result by testing different thresholds.

### History tracking
As we have seen above, `motion()` is useful if you want to remove the still content of your video, only keeping what is different in subsequent frames. Sometimes it is also useful to visualize changes between frames in a different way: layering the last n frames on top of the current one as a video delay. With `history()` you can achieve this, optionally setting the `history length` to the number of past frames you want to see on the current frame (ie. the length of the delay).

Try this:

In [ ]:
mg.history()
mg.show(key='history')

By default `history_length=10`. Let's increase it to 20!

In [ ]:
mg.history(history_length=20)
mg.show(key='history')

### Motion history

To expressively visualize the trajectory of a moving content in a video, you can apply the history process on a motion video. You can do this in one step with `motionhistory()`.

In [ ]:
mg.motionhistory()
mg.show(key='motionhistory')

### Average image
You can also summarize the content of a video by showing the average of all frames in a single image.

In [ ]:
mg.average()
mg.show(key='average')

Embedded in the notebook:

In [ ]:
from IPython.display import Image
average = Image('dance_trim_skip_cb_crop_average.png')
average

### Optical flow
It is also possible to track the direction certain points - or all points - move in a video, this is called 'optical flow'. It has two types: the *sparse optical flow*, which is for tracking a small (sparse) set of points, visualized with an overlay of dots and lines drawing the trajectory of the chosen points as they move in the video.  

In [ ]:
mg.flow.sparse()
mg.show(key='sparse')

Note that sparse optical flow usually works well with slow and continuous movements, where the points to be tracked are not occluded by other objects throughout the course of motion.
Where spare optical flow becomes less reliable, *dense optical flow* often yields more robust results. In dense optical flow the analysis attempts to track the movement of each pixel (or more precisely groups of pixels), colorcoding them with a unique color for each unique direction.

In [ ]:
mg.flow.dense()
mg.show(key='dense')

Sparse optical flow can get confused by too fast movement (ie. too big distance between the locations of a tracked point between two consequtive frames), so it is typically advised not to have a too high `skip` value in the preprocessing stage for it to work properly.
Dense optical flow on the other hand has issues with very slow movement, which sometimes gets below the treshold of what is considered 'a movement' resulting in a blinking video, where the more-or-less idle moments are rendered completely black. If your source video contains such moments, you can try setting `skip_empty=True`, which will discard all the (completely) black frames, eliminating the binking. 

In [ ]:
mg.flow.dense(skip_empty=True)
mg.show(key='dense')

## Chaining

So far our workflow consisted of the following steps:
- 1. Creating an MgObject which loads a video file and optionally applies some preprocessing to it.
- 2. Calling a process on the MgObject.
- 3. Viewing the result.

Something like this:

In [ ]:
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15, skip=3)
mg.motion()
mg.show(key='motion')

This is convenient if you want to apply several different processes on the same input video.

In [ ]:
mg = mgmodule.MgObject('dance.avi', starttime=5, endtime=15, skip=3)
mg.motion()
mg.history()
mg.motionhistory()
mg.average()

The Motion Gestures Toolbox also offers an alternative workflow in case you want to apply a proccess on the result of a previous process. Although `show()` is not really a process (ie. it does not yield a file as a result) it can provide a good example of the use of chaining:

In [ ]:
# this...
mg.motion().show()

In [ ]:
# ...is the equivalent of this!
mg.motion()
mg.show(key='motion')

It also works with images:

In [ ]:
mg.average().show()

But chaining can go further than this. How about reading (and preprocessing) a video, rendering its motion video, the motion history and the average of the motion history, with showing the *_motion_history_average.png* at the end - all as a one-liner?!

In [ ]:
mgmodule.MgObject('dance.avi', skip=4, crop='auto').motion().history().average().show()

In [ ]:
# equivalent without chaining
mg = mgmodule.MgObject('dance.avi', skip=4, crop='auto')
mg.motionhistory()
mh = mgmodule.MgObject('dance_skip_crop_motionhistory.avi')
mh.average()
mh.show(key='average')

Some other examples:

In [ ]:
# rendering and viewing the motion video 
mgmodule.MgObject('dance.avi', skip=4).motion().show()

In [ ]:
# rendering the motion video, the motion history video, and viewing the latter
mgmodule.MgObject('dance.avi', skip=3).motion().history().show()

In [ ]:
# rendering the motion video, the motion average image, and viewing the latter
mgmodule.MgObject('dance.avi', skip=15).motion().average().show()

Chaining can also save time (and space) when designing loops for processing a folder of videos. Here is an example:

In [ ]:
import os
from mgmodule import MgObject as Mg

my_videos_folder = 'C:/Users/User/Desktop/test-videos/'

my_videos = [my_videos_folder + video for video in os.listdir(my_videos_folder) if os.path.splitext(video)[1] in ['.avi', '.mp4', '.mov', '.mkv']]

for video in my_videos:
    print(f'Processing {video}...')
    Mg(video, skip=10).motion().history().average()